In [51]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
import torch.nn as nn
from torchvision import datasets, transforms, models
import os
from torch.utils.data import DataLoader
import torch
from torch.optim import *
from PIL import Image
import torch.nn.functional as F



In [34]:
weights = MobileNet_V2_Weights.DEFAULT
model_mbvnet = mobilenet_v2(weights=weights)

In [48]:
dataset = datasets.ImageFolder('aug_processed_data', transform=transform)

In [35]:
# Freeze backbone
for param in model_mbvnet.parameters():
    param.requires_grad = False

In [38]:
# Transforms
transform = weights.transforms()
base_dir = 'split_data_EfficientNetB0'
train_dataset = datasets.ImageFolder(os.path.join(base_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_mbvnet.classifier[1].parameters(), lr=0.001)


In [40]:
# Replace classifier
num_features = model_mbvnet.classifier[1].in_features
model_mbvnet.classifier[1] = nn.Linear(num_features, len(train_dataset.classes))
model_mbvnet = model_mbvnet.to(device)

In [41]:
# === 3. Training loop ===
def train_model(epochs=10):
    best_val_acc = 0.0
    for epoch in range(epochs):
        model_mbvnet.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model_mbvnet(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        avg_loss = total_loss / len(train_loader)

        model_mbvnet.eval()
        val_correct, val_total, val_loss = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_mbvnet(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)
        if val_acc > best_val_acc:
            best_val_acc = val_acc

        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    print(f"\n🏆 Best Val Accuracy: {best_val_acc:.2f}%")

In [42]:
# === Run training ===
train_model(epochs=10)

Epoch 1 | Train Loss: 0.6992 | Train Acc: 47.50% | Val Loss: 0.6459 | Val Acc: 47.50%
Epoch 2 | Train Loss: 0.7101 | Train Acc: 43.75% | Val Loss: 0.6420 | Val Acc: 60.00%
Epoch 3 | Train Loss: 0.7002 | Train Acc: 52.50% | Val Loss: 0.6416 | Val Acc: 65.00%
Epoch 4 | Train Loss: 0.6963 | Train Acc: 56.25% | Val Loss: 0.6429 | Val Acc: 65.00%
Epoch 5 | Train Loss: 0.6900 | Train Acc: 51.88% | Val Loss: 0.6441 | Val Acc: 62.50%
Epoch 6 | Train Loss: 0.6976 | Train Acc: 50.62% | Val Loss: 0.6468 | Val Acc: 55.00%
Epoch 7 | Train Loss: 0.6984 | Train Acc: 52.50% | Val Loss: 0.6502 | Val Acc: 47.50%
Epoch 8 | Train Loss: 0.6942 | Train Acc: 51.25% | Val Loss: 0.6532 | Val Acc: 45.00%
Epoch 9 | Train Loss: 0.6964 | Train Acc: 54.38% | Val Loss: 0.6565 | Val Acc: 45.00%
Epoch 10 | Train Loss: 0.7021 | Train Acc: 52.50% | Val Loss: 0.6560 | Val Acc: 45.00%

🏆 Best Val Accuracy: 65.00%


Model Testing

In [46]:
def predict_single_image(image_path, model, class_names):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        _, predicted = torch.max(probs, 1)

    print(f"Predicted Class: {class_names[predicted.item()]}")
    print(f"Class Probabilities: {probs.squeeze().numpy()}")

In [52]:
# Assuming dataset = ImageFolder(...)
class_names = dataset.classes  # ['healthy', 'infected']

# Path to one test image
test_image_path_1 = "processed_data/serie infected leaves/infected_05.png"

predict_single_image(test_image_path_1, model_mbvnet, class_names)

Predicted Class: series_infected_leaves_augmented
Class Probabilities: [0.44847256 0.55152744]


Model Evaluation

In [44]:
from sklearn.metrics import classification_report

def evaluate_final_model():
    model_mbvnet.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_mbvnet(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\n📊 Final Evaluation on Validation Set:")
    print(classification_report(all_labels, all_preds, target_names=val_dataset.classes, digits=2))

# Run this after training
evaluate_final_model()


📊 Final Evaluation on Validation Set:
                                  precision    recall  f1-score   support

  serie_healthy_leaves_augmented       0.40      0.20      0.27        20
series_infected_leaves_augmented       0.47      0.70      0.56        20

                        accuracy                           0.45        40
                       macro avg       0.43      0.45      0.41        40
                    weighted avg       0.43      0.45      0.41        40

